# Трансформеры
В этом домашнем задании мы рассмотим использование трансформеров в библиотеке PyTorch. Рассмотрим задачу языкового моделирования. Попробуем генерировать текст нейронной сетью. 

Ссылка на данные - https://drive.google.com/drive/folders/1x1A4ElliUGBPnHladGMwPxPuGxI8Vnpu?usp=sharing

In [1]:
# хороший тон, импортировать все необходимые библиотеки в одной ячейке ;)

import torch
from torch import nn

import numpy as np
import time
import math
from tqdm import trange, tqdm

Что такое языковое моделирование? Это предсказание вероятности следующего токена (слова или буквы) на основе предыдущих токенов. Математически это можно описать так:

$$P(x_i|x_1, x_2 , ... , x_{i-1})$$ 

Последовательность $$ x_1, x_2, ... x_{i-1} $$ называют контекстом.

## Задание 0 (0 баллов, но сделать нужно)
Проставьте знаки неравенств, исходя из вашего опыта:
$$ P(раму | мама, мыла) * P(папу | мама, мыла) $$
$$ P(столу | дорога, ложка, к) * P(обеду | дорога, ложка, к) $$
$$ P(Евпатий | меня, зовут) * P(Ваня | меня, зовут) $$
$$ P(журналы | я, часто ,читаю) * P(комиксы | я, часто ,читаю) $$
Попробуйте объяснить выбор для каждого из примеров.

Ответ : > , так как есть такой стишок 

< , ложка к обеду звучит как-то логичнее

< , Ваня более часто встречающееся имя чем Евпатий

< , для молодого поколения очевиднее комиксы, чем журналы

Если для некоторых из примеров проставить знаки достаточно просто, то на некоторые сложно сказать, какой овтет верный. Мы принимаем решение для данного задания исходя их опыта использования русского языка. Мы много читали на русском и слушали огромное количество русской речи. Обучение языковых моделей происходит по схожему принципу. 

Мы хотим показать модели столько текстов, сколько можем и надеемся, что она наберется достаточно опыта, чтобы расставлять такие знаки неравества максимально схоже с человеком.

## Задание 1 (0.5 балла)
Мы будем обучать языковую модель для предсказания следущей буквы. Такие языковые модели применяются в распозновании речи, так как предоставляют дополнительную информацию акустической модели при выборе следующего символа. Для начала, откройте файл с данными, посмотрите, какие символы входят в тексты, сколько их. Уберите из текста все символы переноса на новую строку и табуляцию.

In [2]:
path = 'small_corp_for_test.txt'
file = open(path, 'r')
data = file.readlines()
file.close()
len(data)

700000

In [3]:
from collections import Counter
import itertools

In [4]:
data = [text.split() for text in data]

In [ ]:
letters = list()
for string in data:
    for word in string:
        word = list(word)
        for char in word:
            letters.append(char)

In [ ]:
my_dict = Counter(letters)

In [ ]:
my_dict

Counter({'-': 43618,
         'а': 2366842,
         'б': 384499,
         'в': 1130062,
         'г': 402608,
         'д': 899550,
         'е': 2028774,
         'ж': 249344,
         'з': 355889,
         'и': 1501660,
         'й': 289713,
         'к': 795405,
         'л': 915552,
         'м': 878018,
         'н': 1582468,
         'о': 2779376,
         'п': 757949,
         'р': 1055739,
         'с': 1401746,
         'т': 1985934,
         'у': 848660,
         'ф': 60354,
         'х': 147781,
         'ц': 117824,
         'ч': 370993,
         'ш': 187256,
         'щ': 66697,
         'ъ': 5406,
         'ы': 403604,
         'ь': 538119,
         'э': 162631,
         'ю': 149791,
         'я': 555131,
         'ё': 76849})

In [ ]:
sum = 0
for value in my_dict.values():
    sum += value
print(f"Total number of letters is {sum}")

Total number of letters is 25495842


## Задание 2 (0.5 балла)
Для обучения модели требуется сначала подготовить текст в подходящий для нейросети вид. Важно также отметить, что нужно добавить два токена start и end, которые отвечают за начало и конец текста. Используйте [ и ] для этой задачи. Также нам нужен токен pad, чтобы заполнять им текст до требуемой длинны для формирования батча.

Реализуйте метод preprocess класса Preprocessor. Он должен принимать на вход текст и длинну текста, которую мы ожидаем получить на выходе. Текст должен быть переведен в нижний регистр, в конец текста добавляется требуемое число pad токенов, далее текст векторизуется (каждому символу ставится свое число). Вернуть требуется два вектора. Полученный результат без последнего токена (на нем будем обучаться) и полученный результат без первого токена (целевые метки при обучении).

In [5]:
class Preprocessor:
    def __init__(self):
        self.alphabet = '_добсркгаупитнезчмфяжлйвцыэь-шхющёъ][ '
        self.token2ind = {}
        self.ind2token = {}
        for i in range(len(self.alphabet)):
            self.token2ind[self.alphabet[i]] = i
            self.ind2token[i] = self.alphabet[i]
        
    
    def preprocess(self, text, window_size):
        answer = []
        text = text.lower() + "_" * (window_size - len(text))
        for symbol in text:
            answer.append(self.token2ind[symbol])
        return np.array(answer[:-1]), np.array(answer[1:])

## Задание 3 (0.5 балла)
Так как мы решили, что текст будет начинаться токеном [ и заканчиваться токеном ], данные нужно поправить. Реализуйте эту идею, добавьте данные токены в ваши тексты.

In [6]:
for i in range(len(data)):
    data[i] = '[' + " ".join(data[i]) + ']'

In [7]:
data

['[добро]',
 '[кого]',
 '[капитан]',
 '[нет]',
 '[зачем]',
 '[что происходит]',
 '[что такое]',
 '[рассказ]',
 '[никому]',
 '[ну что]',
 '[кто]',
 '[я укажу]',
 '[исполняй]',
 '[ждет]',
 '[он думал]',
 '[в броне]',
 '[отец]',
 '[быстро]',
 '[ну жил]',
 '[что]',
 '[здоров]',
 '[что с тобой]',
 '[иортэ]',
 '[как]',
 '[ну как]',
 '[ну что ж]',
 '[ах да]',
 '[сударь]',
 '[утонул]',
 '[меня]',
 '[спросил он]',
 '[ну-ка]',
 '[увы]',
 '[но]',
 '[простите]',
 '[вот-вот]',
 '[быстро]',
 '[ждать]',
 '[зачем]',
 '[зачем]',
 '[по пути]',
 '[он здесь]',
 '[ив]',
 '[а другой]',
 '[да нет же]',
 '[случайно]',
 '[ну что ж]',
 '[а как же]',
 '[капри]',
 '[нет]',
 '[я просто]',
 '[уже день]',
 '[сонька]',
 '[ого]',
 '[э-э-э]',
 '[за что]',
 '[как]',
 '[а]',
 '[вот видишь]',
 '[боже]',
 '[а тебе]',
 '[кто]',
 '[не было]',
 '[спросил он]',
 '[не ври]',
 '[ладно]',
 '[крикнул гарри]',
 '[сказал я]',
 '[спросил я]',
 '[посмотри]',
 '[стрела]',
 '[ждешь чего]',
 '[нагнись]',
 '[когда]',
 '[а я]',
 '[марина]'

## Задание 4 (0.5 балла)
Так как мы не располагаем большими мощностями, то давайте ограничим максимальную длинну текста. Вы можете менять этот порог и тем самым уменьшать кол-во текстов в вашей выборке и увеличивая тем самым скорость обучения. Начнем же мы с 128. 
Выберите порог и оставьте только те тексты, длина которых не превосходит данный порог.

Далее разбейте тексты на train и test, перемешайте тексты при разбиении, размер тестовой выборки должен быть 15% от общего числа текстов. 

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
THRESHOLD = 128
for text in data[:]:
    if len(text) > THRESHOLD:
        data.remove(text)

In [10]:
len(data)

683439

In [11]:
train, test = train_test_split(data, test_size=0.15, random_state=12345, shuffle=True)

## Задание 5 (2 балла)
Напишем датасет. На вход датасету передается набор текстов, объект класса Preprocessor и размер окна, который вы выбрали в прошлом задании.
Реализуйте методы __len__ и __getitem__.

In [12]:
class TextDataset(torch.utils.data.Dataset):
    
    def __init__(self, x, preproc, win_size = 128):
        self.x = x
        self.preproc = preproc
        self.win_size = win_size
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return preproc.preprocess(self.x[idx], self.win_size)

In [13]:
preproc = Preprocessor()
train_dataset = TextDataset(train, preproc)
test_dataset = TextDataset(test, preproc)

## Задание 6 (2 балла)
Напишем модель. Класс для реализации positional encoding реализован за вас, он нужен, чтобы модель могла после получения эмбедингов понимать, на каком месте какой токен находится.

Заполните пропуски в классе модели. Гипперпараметры модели вам предлагается подобрать самостоятельно. Рекомендуется использовать не более 6 слоев в трансформере. В декореде испоьлзуйте две линейных слоя с функцией активации ReLU между ними.

## Задание 6_1 (0 баллов, но надо ответить!)
При обучении языковой модели на основе трансформеров мы используем маскирование символов (как мы это делаем - уже реализовано). Напишите, почему мы это делаем? Почему это так важно?

С помощью маски мы делаем так, что модель не подглядывает в ответы, то есть на текущем символе мы не видим то, что будет после него. И за счет этого мы смотрим только на то, что было раньше до этого символа, и таким образом обучаемся.

In [14]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [41]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads):
        super(LanguageModel, self).__init__()
        self.emb = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model
        self.pe = PositionalEncoding(d_model)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model, n_heads)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers=5) 
        self.decoder = nn.Sequential(
            nn.Linear(d_model, 512),
            nn.ReLU(),
            nn.Linear(512, vocab_size))
    
    def forward(self, x, src_mask): #src_mask
        x = self.pe(self.emb(x))
        x = x.transpose(1, 0)
        x = self.transformer_encoder(x, src_mask) # transformer encoder with mask
        x = self.decoder(x)
        return x.transpose(1, 0)
    
    def generate_square_subsequent_mask(self, sz):
        # А вот и то самое маскирование
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [42]:
model = LanguageModel(len('_добсркгаупитнезчмфяжлйвцыэь-шхющёъ][ '), 256, 8)

## Задание 7 (2,5 балла)
Финишная прямая. Давайте реализуем класс для обучения модели и ее валидации. Следуйте указаниям в коде и заполните недостающие фрагменты в коде.

In [43]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        
        self.train_batch_size = 64
        self.test_batch_size = 64
        
        self.train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=self.train_batch_size)
        self.test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=self.test_batch_size)
        self.train_dataloader_size = len(self.train_dataloader)
        self.test_dataloader_size = len(self.test_dataloader)
        
        self.device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
        self.criterion = nn.CrossEntropyLoss(ignore_index=0) # используйте CrossEntrophyLoss, передайте в качетсве параметра 
                             # ignore index индекс символа _, чтобы модель не штрафовалась за то
                             # что идет после закрывающего токена
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001) # lr=0.01
        
        self.steps_to_print = 250
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        
        for batch in tqdm(self.train_dataloader):
            x, y = batch
            # YOUR CODE HERE
            it += 1
            step += 1

            x = x.to(self.device) 
            y = y.to(self.device) 

            sequence_length = x.size(1)
            src_mask = self.model.generate_square_subsequent_mask(sequence_length).to(self.device)
            predicted = self.model(x, src_mask)

            loss = self.criterion(predicted.permute(0, 2, 1), y)

            # Update weights
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            
            counted_loss += loss.detach().item()
            # реализуйте шаги обучения модели
            # сохраняйте значение ошибки в переменную counted_loss
            
            ################
            
            
            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss/self.steps_to_print)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        for batch in tqdm(self.test_dataloader):
            x, y = batch
            step += 1
            it += 1
            # YOUR CODE HERE
            
            # реализуйте шаги для теста модели
            # помните, что данный метод уже запускается из 
            # блока with torch.no_grad(), а потому 
            # повторно его использовать не нужно
            
            x = x.to(self.device)
            y = y.to(self.device)
            sequence_length = y.size(1)
            src_mask = self.model.generate_square_subsequent_mask(sequence_length).to(self.device)
            predicted = self.model(x, src_mask)
            loss = self.criterion(predicted.permute(0, 2, 1), y)
            counted_loss += loss.item()
            
            ################
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss/self.steps_to_print)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

Что такое ppl? Перплексия. Ее можно интерпретировать как меру "удивленности" модели нужному символу. Чем меньше данная величина, тем лучше, ведь это значит, что модель если и сделала неправильный выбор, то не сильно удивлена своей ошибке.

Проведите несколько экспериментов, посмотрите, при каких гипперпараметрах значение перплексии минимально.

## Задание 8 (0.5 балла)
Запустите обучение на нескольких эпохах. Ориентируйтесь на ваши вычислительные мощности и время работы. Вы всегда можете посчитать, сколько секунд уходит на один батч.

In [44]:
trainer = Trainer(model, train_dataset, test_dataset)

trainer.train(2)

  3%|▎         | 250/9077 [01:10<41:11,  3.57it/s]

Train epoch 1 | Step 250/9077 | Counted loss 2.547689658164978 | ppl 12.777549148808081 | time 70.6369137763977 | 


  6%|▌         | 500/9077 [02:20<40:36,  3.52it/s]

Train epoch 1 | Step 500/9077 | Counted loss 1.9761751217842103 | ppl 7.215093287366765 | time 70.23758053779602 | 


  8%|▊         | 750/9077 [03:31<38:50,  3.57it/s]

...

 94%|█████████▎| 1501/1602 [02:38<00:10,  9.34it/s]

Validate epoch 1 | Step 1500/1602 | Counted loss 0.6260684146881104 | ppl 3.4978091990276554 | time 13.218628406524658 | 


100%|██████████| 1602/1602 [02:49<00:00,  9.44it/s]


  0%|          | 18/9077 [00:05<44:32,  3.39it/s]


KeyboardInterrupt: ignored

Так как модель обучается долго, а ошибка почти не падает, то я прервал обучение после 1й эпохи. Несмотря на это, она уже выдает более менее осмысленные тексты (видно дальше)

## Задание 9 (1 балл)
Итак, давайте попробуем погенерировать текст нашей сеткой. Закончите функцию по генерации текста. Попробуйте сгенерировать какой-нибудь текст. Помните, что если вы хотите генерировать текст с нуля, то вы должны передать в качестве текста только токен start.
Прекратите генерировать текст, если модель выдала токен end или длинна текста больше 150.

In [60]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

Максимальную длинну текста пришлось ограничить до 50, так как при большей длине кончается память на гпу из-за большого количество вызовов рекуррентной функции генерации текста

In [101]:
def generate_text(text):
    x = []
    for letter in text:
        x.append(preproc.token2ind[letter])
    x = torch.from_numpy(np.array(x)).to(device)
    sequence_length = len(x)
    src_mask = model.generate_square_subsequent_mask(sequence_length).to(device)

    pred = model(x, src_mask)
    ind = pred.topk(1)[1].view(-1)[-1].item()
    
    text += preproc.ind2token[ind]
    
    if len(text) > 50 or text[-1] == "]":
        return text
    else:
        return generate_text(text)

In [102]:
generate_text('[здравствуйте')

'[здравствуйте меня зовут антон слушаю вас]'

In [103]:
generate_text('[')

'[ну вот так сейчас я посмотрю вот вот так по поводу'

In [105]:
generate_text('[глубинное обучение')

'[глубинное обучение по поводу подробнее подразделен'

In [106]:
generate_text('[до свидания')

'[до свидания спасибо до свидания]'

## Задание 10* (Задание - бонус, 5 баллов за реализацию при условии, что сделаны прошлые задания)
Давайте вспомним, что такое transfer learning. Мы хотим использовать уже предобученные эмбединги для нашей сети, чтобы наша сеть обучалась быстрее. Давайте попробуем обучить новую модель на уровне слов, а не символов, но для упрощения задачи используем предобученный слой из библиотеки Natasha, а вернее, ее блок Navec.

[Изучите](https://github.com/natasha/navec) то, как вставить слой в вашу нейронную сеть.

Теперь мы хотим, чтобы на вход модели подавались слова, модифицируйте ваш датасет. Возвращайте теперь номер слова в словаре navec.

In [ ]:
class TextDataset_Navec(torch.utils.data.Dataset):
    
    def __init__(self, x, win_size = 128):
        # YOUR CODE HERE
        self.navec = ...
        ################
    
    def __len__(self):
        # YOUR CODE HERE
        ################
    
    def __getitem__(self, idx):
        # YOUR CODE HERE
        ################

Немного модифицируем модель. Теперь нам не нужны слои с трансформером, так как весь механизм внимания уже заложен в ембедингах. Давайте попробуем просто пройтись линейной головой над эмбедингами. Выберите параметры самостоятельно.

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self):
        super(LanguageModel, self).__init__()
        self.emb_navec = ...
        self.head = ...
    
    def forward(self, x):
        x = ... # emb
        x = ... # head
        return x

Теперь дело за малым! Надо немного модифицировать класс обучения, так как мы не используем маскирование, после чего можно приступить к тесту!

In [ ]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        
        self.train_batch_size = 64
        self.test_batch_size = 64
        
        self.train_dataloader = ...
        self.test_dataloader = ...
        self.train_dataloader_size = ...
        self.test_dataloader_size = ...
        
        self.device = 'cuda:0'
        self.criterion = ... 
        
        self.optimizer = ...
        
        self.steps_to_print = 1000
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        
        for batch in self.train_dataloader:
            x, y = batch
            # YOUR CODE HERE
            
            # реализуйте шаги обучения модели
            # сохраняйте значение ошибки в переменную counted_loss
            
            ################
            
            
            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        for batch in self.test_dataloader:
            x, y = batch
            
            # YOUR CODE HERE
            
            # реализуйте шаги для теста модели
            # помните, что данный метод уже запускается из 
            # блока with torch.no_grad(), а потому 
            # повторно его использовать не нужно
            
            ################
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

Запустите обучение. 

In [ ]:
# YOUR CODE HERE
###############